In [75]:
import pandas as pd
import numpy as np

nips = pd.read_csv('./NIPS/nips.txt',delimiter=' ')
nips = nips.pivot(columns='wordid', index='docid', values='count')
nips = nips.fillna(value=0)
nips = np.add(nips,0.1)

badcols = list(np.flatnonzero(np.sum(nips.values, 1)<1000))
allcols = list(range(1,1491))
cols = [x for x in allcols if x not in badcols]
nips = nips.iloc[:,cols]
nips.columns = list(range(0,nips.shape[1]))
#nips = nips.add(1)
nips_data = nips.values

print(str(nips.shape[0]) + " unique documents")
print(str(nips.shape[1]) + " unique words")
nips.head()

1500 unique documents
1490 unique words


,0,1,2,3,4,5,6,7,8,9,...,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489
docid,,,,,,,,,,,,,,,,,,,,,
1,1.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,1.1,0.1,0.1,0.1,4.1,16.1,1.1,4.1,3.1
2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,1.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [76]:
def e_step(data,mu,k,pi):
    
    # data is a dxn numpy array of word counts
    # mu is a kxd numpy array of cluster centroids
    # k is the cluster number
    # pi is a kx1 array of cluster membership probabilities
    
    n = data.shape[0]
    d = data.shape[1]
    
    h1 = np.multiply(data,data)
    h1 = np.matmul(h1,np.ones((d,k)))
    
    h2 = np.matmul(np.ones((n,d)), np.multiply(mu,mu).T)
    
    h3 = np.matmul(data,mu.T)
    
    H = -0.5*(np.subtract(np.add(h1,h2),2*h3))
    #print(np.mean(H))

    P = np.matmul(np.ones((n,1)),pi.T)
    #print(np.mean(P))
    
    E = np.multiply(np.exp(H),P)
    F = np.matmul(E,np.ones((k,k)))
    F = np.add(F,0.01)
    W = np.divide(E,F)
    return(W)


def m_step(W,data):
    # W is a nxk array of document likelihoods (given cluster)
    # data is a dxn array of word counts
    n = data.shape[0]
    d = data.shape[1]
    
    D = np.matmul(W.T,data)
    G = np.matmul(W.T,np.ones((n,d)))
    mu_new = np.divide(D,G)
    return(mu_new)

def m_step_log(W,data,pis,alpha):
    n = data.shape[0]
    d = data.shape[1]
    pi_new = np.log(np.matmul(W.T,np.ones((n,1))))
    pi_new = np.subtract(pi_new, np.log(n))
    W = np.multiply(alpha,W)
    mu_new = np.matmul(W.T,data)
    mu_new = np.divide(mu_new,np.matmul(W.T,np.ones((n,d))))
    return(mu_new)
    
def evalulate(mu_old,mu_new,threshold):
    diff = np.subtract(mu_new,mu_old)
    if np.linalg.norm(diff) <= threshold:
        return(True)
    else:
        return(False)
    

    

In [77]:
## Initialize
k = 3
d = nips_data.shape[1]
n = nips_data.shape[0]
topic_centers = np.random.random(k)
mu = topic_centers.reshape((k,1))
mus = np.random.random(k*d)
mus = mus.reshape((k,d))
pi = np.ones((k,1))*1/k

counter = 1
#while counter<10:
W = e_step(data=nips_data,k=3,mu=mus,pi=pi)

#inspect the weights we got back
print("Each of these number should be between 0 and 1500 (#number of documents)")
print("When all summed together, they sholuld be roughly 1500")
print("sum of document weights to first topic:", sum(W.T[0]))
print("sum of document weights to second topic:", sum(W.T[1]))
print("sum of document weights to third topic:", sum(W.T[2]))
print(np.mean(W))

mu_new = m_step(data=nips_data,W=W)
print(np.mean(mu_new))
mu_new = m_step_log(data=nips_data,W=W,pis=pi,alpha=10)
print(np.mean(mu_new))

#print("Iteration " + str(counter))
#print(np.mean(mu_new))
#counter += 1

1.085839084001476e-79
0.10021891387010154
0.10021891387010154
